In [ ]:
# Setup
import os
import json
from dotenv import load_dotenv
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from llama_stack_client.lib.agents.event_logger import EventLogger

# Load configuration
load_dotenv('config.env')
base_url = os.environ.get('LLAMA_STACK_URL', 'http://localhost:8321')
model_id = os.environ.get('LLM_MODEL_ID', 'r1-qwen-14b-w4a16')
temperature = float(os.environ.get("TEMPERATURE", 0.0))
max_tokens = int(os.environ.get("MAX_TOKENS", 4096))

print(f"🔗 Connecting to: {base_url}")
print(f"🤖 Using model: {model_id}")

# Create client
client = LlamaStackClient(base_url=base_url)
print("✅ Client created")


In [ ]:
# Create ReActAgent
print("🤖 Creating ReActAgent...")

# Sampling parameters
sampling_params = {
    "strategy": {"type": "greedy"},
    "max_tokens": max_tokens,
    "temperature": temperature,
}

# Create agent
agent = ReActAgent(
    client=client,
    model=model_id,
    instructions="""You are a Jira assistant. When asked to create a Jira issue, generate the correct tool call parameters.
    Use jira_create_issue tool with project_key, summary, issue_type, description, and additional_fields.
    Always provide additional_fields as a dictionary with priority and labels.""",
    tools=["mcp::atlassian"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
    sampling_params=sampling_params,
)

print("✅ ReActAgent created")


In [ ]:
# Function to execute ReActAgent + Manual Tool Call
def create_jira_with_agent(prompt, session_name="jira-agent"):
    """Use ReActAgent to generate tool parameters, then execute manually"""
    print(f"\n🧪 Creating Jira issue with ReActAgent: {prompt}")
    print("="*60)
    
    try:
        # Create session
        session_id = agent.create_session(session_name)
        print(f"✅ Session created: {session_id}")
        
        # Get agent response (non-streaming for easier parsing)
        response = agent.create_turn(
            messages=[{"role": "user", "content": prompt}],
            session_id=session_id,
            stream=False,
            toolgroups=["mcp::atlassian"]
        )
        
        print("🤖 ReActAgent response:")
        print(f"Response: {response.output_message.content}")
        
        # Parse the JSON response
        import json
        agent_data = json.loads(response.output_message.content)
        
        print(f"Thought: {agent_data.get('thought', 'N/A')}")
        print(f"Action: {agent_data.get('action', 'N/A')}")
        print(f"Answer: {agent_data.get('answer', 'N/A')}")
        
        # Extract tool parameters from agent response
        action = agent_data.get('action', {})
        if action.get('tool_name') == "jira_create_issue":
            print("\n🔧 Executing tool call manually...")
            
            # Convert tool_params to kwargs format
            kwargs = {}
            for param in action.get('tool_params', []):
                kwargs[param['name']] = param['value']
            
            # Add default additional_fields if not provided
            if "additional_fields" not in kwargs:
                kwargs["additional_fields"] = {}
            
            print(f"Tool parameters: {kwargs}")
            
            # Execute the tool call
            result = client.tool_runtime.invoke_tool(
                tool_name=action['tool_name'],
                kwargs=kwargs
            )
            
            print("✅ Tool executed successfully!")
            print(f"Result: {result}")
            return result
        else:
            print(f"❌ No valid jira_create_issue action found. Got: {action.get('tool_name', 'None')}")
            return None
            
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None


In [ ]:
# Test 1: Simple Jira Issue Creation
print("🧪 Test 1: Simple Jira Issue Creation")
print("="*40)

result1 = create_jira_with_agent(
    "Create a Jira issue in the KAN project with summary 'ReActAgent Working Test' and type 'Task'",
    "test-simple"
)

if result1:
    print("🎉 Test 1 SUCCESS: Issue created!")
else:
    print("❌ Test 1 FAILED: Issue not created")


In [ ]:
# Test 2: OOM Error Incident Creation
print("\n🧪 Test 2: OOM Error Incident Creation")
print("="*40)

result2 = create_jira_with_agent(
    """Create a Jira incident for a pod failing due to OOM error in the KAN project:
    - Summary: 'Pod failing due to OOM error'
    - Issue Type: 'Incident'
    - Description: 'Pod experiencing Out of Memory errors causing application failures'
    - Priority: High
    - Labels: ['oom-error', 'pod-failure', 'high-priority']""",
    "test-oom"
)

if result2:
    print("🎉 Test 2 SUCCESS: OOM incident created!")
else:
    print("❌ Test 2 FAILED: OOM incident not created")


In [ ]:
# Test 3: Verify Issues Were Created
print("\n🧪 Test 3: Verify Issues Were Created")
print("="*40)

try:
    # Search for recently created issues
    search_result = client.tool_runtime.invoke_tool(
        tool_name="jira_search",
        kwargs={"jql": "project = KAN ORDER BY created DESC"}
    )
    
    print("✅ Search completed!")
    
    # Check if our test issues were created
    search_text = str(search_result)
    if "ReActAgent Working Test" in search_text:
        print("🎉 SUCCESS: ReActAgent Working Test issue found!")
    else:
        print("⚠️  ReActAgent Working Test issue not found")
        
    if "OOM error" in search_text:
        print("🎉 SUCCESS: OOM error incident found!")
    else:
        print("⚠️  OOM error incident not found")
        
    print(f"\nRecent issues in KAN project:")
    print(search_result)
        
except Exception as e:
    print(f"❌ Search failed: {e}")
